# ATL06-SR Demo with Visualization
David Shean  
December 8, 2020

Testing current SlideRule functionality for Everest and Grand Mesa sites

In [ ]:
import sys
import os
import logging
import time
import json
import pandas as pd
import numpy as np
import cartopy
import matplotlib.pyplot as plt
from datetime import datetime
from sliderule import icesat2

In [ ]:
def process_atl06_algorithm(parms, asset):

    # Latch Start Time
    perf_start = time.perf_counter()

    # Request ATL06 Data
    rsps = icesat2.atl06p(parms, asset)

    # Latch Stop Time
    perf_stop = time.perf_counter()

    # Build DataFrame of SlideRule Responses
    df = pd.DataFrame(rsps)

    # Display Statistics
    perf_duration = perf_stop - perf_start
    print("Completed in {:.3f} seconds of wall-clock time".format(perf_duration))
    print("Reference Ground Tracks: {}".format(df["rgt"].unique()))
    print("Cycles: {}".format(df["cycle"].unique()))
    print("Received {} elevations".format(len(df)))

    # Return DataFrame
    return df

In [ ]:
# Configure Logging #
logging.basicConfig(level=logging.INFO)

In [ ]:
# Region of Interest #
region_filename = 'everest.json'
#region_filename = 'ngozumpa.json'
with open(region_filename) as regionfile:
   # region = json.load(regionfile)["region"]
    region = json.load(regionfile)["region"]

In [ ]:
#Can specify region manually
#Grand Mesa
"""
region = [ {"lon": -108.3435200747503, "lat": 38.89102961045247},
               {"lon": -107.7677425431139, "lat": 38.90611184543033}, 
               {"lon": -107.7818591266989, "lat": 39.26613714985466},
               {"lon": -108.3605610678553, "lat": 39.25086131372244},
               {"lon": -108.3435200747503, "lat": 38.89102961045247} ]
"""

In [ ]:
# Set URL #
url = ["44.227.4.67"]

# Set Asset #
asset = "atlas-s3"
#asset = "atl06-s3"

In [ ]:
# Configure SlideRule
icesat2.init(url, True)

In [ ]:
# Build ATL06 Request
parms = {
    "poly": region,
    "srt": icesat2.SRT_LAND,
    "cnf": icesat2.CNF_SURFACE_HIGH,
    "ats": 10.0,
    "cnt": 10,
    "len": 40.0,
    "res": 20.0,
    "maxi": 1
}

In [ ]:
out_fn = 'everest.pkl'
if not os.path.exists(out_fn):
    # Get ATL06 Elevations
    atl06 = process_atl06_algorithm(parms, asset)
    atl06.to_pickle(out_fn)
else:
    atl06 = pd.read_pickle(out_fn)

In [ ]:
atl06.shape

In [ ]:
atl06

In [ ]:
#Currently using astropy for GPS time handling
from astropy.time import Time

def time_convert(df):
    print("Converting GPS time to Pandas DateTime")
    
    #Some early ATL03 delta_time values use different reference epoch, need to correct these 
    t_ref1 = Time('2018-01-06T00:00:00', format='isot')
    t_ref2 = Time('1980-01-06T00:00:00', format='isot')
    t_off = t_ref1.gps - t_ref2.gps
    
    df.loc[df['delta_time'] < t_ref1.gps, 'delta_time'] += t_off 
    
    temp = Time(t_ref2.gps + df['delta_time'], format='gps')
    
    df['utc'] = pd.to_datetime(temp.isot)
    
    print("Converting timestamp to decyear")
    df['decyear'] = temp.decimalyear
    
    print("Resetting index and sorting")
    df.set_index('utc', inplace=True)
    df.sort_index(inplace=True)

In [ ]:
from pyproj import Transformer
def add_xy(df):
    #Add x and y coords in web mercator (for now)
    crs = "EPSG:3857" #Web mercator
    transformer = Transformer.from_crs("EPSG:4326", crs)
    print("Reprojecting to %s" % crs)
    df['x'], df['y'] = transformer.transform(df['lat'].values, df['lon'].values)

In [ ]:
time_convert(atl06)

In [ ]:
add_xy(atl06)

In [ ]:
atl06.head()

In [ ]:
import geopandas as gpd
import contextily as cx

In [ ]:
atl06_gdf = gpd.GeoDataFrame(atl06, geometry=gpd.points_from_xy(atl06['lon'], atl06['lat']), crs='EPSG:4326')

In [ ]:
n = 100000
t = atl06_gdf.sample(n=n)

In [ ]:
f, axa = plt.subplots(1,2, figsize=(15,5))
plot_kw = {'legend':True, 'markersize':1}
t.plot(ax=axa[0], column='h_mean', **plot_kw)
t.plot(ax=axa[1], column='decyear', **plot_kw)
cx.add_basemap(axa[0], crs=t.crs.to_string(), source=cx.providers.Stamen.Terrain)
cx.add_basemap(axa[1], crs=t.crs.to_string(), source=cx.providers.Stamen.Terrain)

In [ ]:
import hvplot.pandas
from holoviews.element.tiles import EsriImagery, StamenTerrain

In [ ]:
#map_tiles = EsriImagery()
map_tiles = StamenTerrain()

In [ ]:
kw = {'cmap':'inferno', 'width':500, 'height':500, 'hover':False, 'data_aspect':1, 'alpha':1.0}
kw['colorbar'] = True
#kw['s'] = 1
#kw['aggregator'] = 'mean'
#kw['rasterize'] = False
#Using datashade=True drops colorbar
kw['datashade'] = True
#kw['dynamic'] = True

In [ ]:
map_tiles * atl06.hvplot.scatter(x='x', y='y', c='h_mean', title='%s: %s' % ('ATL06-SR', 'h_mean'), **kw) + \
map_tiles * atl06.hvplot.scatter(x='x', y='y', c='dec_year', title='%s: %s' % ('ATL06-SR', 'dec_year'), **kw)

## Next Steps
* Retrieve and plot original ATL06 points
* Zoom for comparisons of ATL06-SR vs. ATL06